In [ ]:
import lseg.data as ld
from lseg.data.content import historical_pricing
from lseg.data.content.historical_pricing import Intervals

import asyncio
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from pathlib import Path

import seaborn as sns
import matplotlib.pyplot as plt


ld.open_session()

In [ ]:
def save_data_as_csv(ticker,out_path, count=3000):
    response = historical_pricing.summaries.Definition(
        universe=ticker,
        interval=Intervals.DAILY,
        count=count,
        fields=["BID", "ASK"]
    ).get_data()
    
    response.data.df.to_csv(out_path)

In [ ]:
# Read the CSV containing the symbols and sectors
symbols_df = pd.read_csv("sp500_symbols_sectors.csv")
symbols = symbols_df['Symbol'].tolist()
errors = []

# Loop through symbols with a progress bar
for symbol in tqdm(symbols, desc="Processing symbols"):
    try:
        save_data_as_csv(symbol, f'data_dl/{symbol}.csv')
    except Exception as E:
        try:
            save_data_as_csv(f"{symbol}.O", f'data_dl/{symbol}.O.csv')
        except Exception as E:
            try:
                save_data_as_csv(f"{symbol}.K", f'data_dl/{symbol}.K.csv')
            except Exception as E:
                errors.append(symbol)
                print(f"error on {symbol}")

# Print the errors
print(f"errors {errors}")

In [ ]:
# Read the CSV containing the symbols and sectors
symbols_df = pd.read_csv("sp500_symbols_sectors.csv")
symbols = symbols_df['Symbol'].tolist()
industries = symbols_df['GICS Sector'].tolist()

sym_ind = zip(symbols, industries)

sym_ind_true = {}

errors = []

def lookup(symbol):
    response = historical_pricing.summaries.Definition(
        universe=symbol,
        interval=Intervals.DAILY,
        count=1,
    ).get_data()

# Loop through symbols with a progress bar
for symbol, industry in tqdm(sym_ind, desc="Processing symbols"):
    try:
        lookup(symbol)
        sym_ind_true[symbol] = industry
    except Exception as E:
        try:
            lookup(f"{symbol}.O")
            sym_ind_true[f"{symbol}.O"] = industry
        except Exception as E:
            try:
                lookup(f"{symbol}.K")
                sym_ind_true[f"{symbol}.K"] = industry
            except Exception as E:
                errors.append(symbol)
                print(f"error on {symbol}")

# Print the errors
print(f"errors {errors}")

In [ ]:
from lseg.data.content import fundamental_and_reference

definition = fundamental_and_reference.Definition(
    universe="IBM.N",
    fields=["TR.Revenue"]
)

definition.fields

In [ ]:
# Define the directory containing individual stock CSVs
DATA_DIR = Path("data_dl")
BID_OUTPUT = "aggregated_bid.csv"
ASK_OUTPUT = "aggregated_ask.csv"

def get_stock_symbol(file_path):
    """
    Extracts the stock symbol from the filename.
    Assumes filenames are in the format 'SYMBOL.csv'.
    """
    return file_path.stem  # 'AAPL.csv' -> 'AAPL'

def read_stock_data(file_path):
    """
    Reads a stock CSV and returns a DataFrame with 'Date' as datetime.
    Ensures required columns are present.
    """
    try:
        df = pd.read_csv(file_path, parse_dates=['Date'])
        if not {'Date', 'BID', 'ASK'}.issubset(df.columns):
            raise ValueError(f"Missing required columns in {file_path.name}")
        df = df.sort_values('Date').reset_index(drop=True)
        return df
    except Exception as e:
        raise ValueError(f"Error reading {file_path.name}: {e}")

def aggregate_stock_data():
    """
    Aggregates BID and ASK data from individual stock CSVs into two separate CSVs.
    Performs an outer join on dates to include all available dates.
    Missing data for a stock on a particular date is represented as NaN.
    """
    # Locate all CSV files in the data directory
    csv_files = list(DATA_DIR.glob("*.csv"))
    
    if not csv_files:
        raise FileNotFoundError(f"No CSV files found in {DATA_DIR}")
    
    # Initialize empty DataFrames for BID and ASK with 'Date' as the key
    aggregated_bid_df = pd.DataFrame()
    aggregated_ask_df = pd.DataFrame()
    
    # To keep track of errors
    error_symbols = []
    
    # Process each CSV file with a progress bar
    for file_path in tqdm(csv_files, desc="Aggregating stock data"):
        symbol = get_stock_symbol(file_path)
        try:
            df = read_stock_data(file_path)
            
            # Select relevant columns and rename them to the stock symbol
            bid_series = df[['Date', 'BID']].rename(columns={'BID': symbol})
            ask_series = df[['Date', 'ASK']].rename(columns={'ASK': symbol})
            
            # Merge with the aggregated DataFrames using an outer join on 'Date'
            if aggregated_bid_df.empty:
                aggregated_bid_df = bid_series
                aggregated_ask_df = ask_series
            else:
                aggregated_bid_df = pd.merge(aggregated_bid_df, bid_series, on='Date', how='outer')
                aggregated_ask_df = pd.merge(aggregated_ask_df, ask_series, on='Date', how='outer')
        
        except Exception as e:
            error_symbols.append(symbol)
            print(f"Error processing {symbol}: {e}")
    
    # Sort the aggregated DataFrames by 'Date'
    aggregated_bid_df = aggregated_bid_df.sort_values('Date').reset_index(drop=True)
    aggregated_ask_df = aggregated_ask_df.sort_values('Date').reset_index(drop=True)
    
    # Save the aggregated DataFrames as CSVs
    aggregated_bid_df.to_csv(BID_OUTPUT, index=False)
    aggregated_ask_df.to_csv(ASK_OUTPUT, index=False)
    
    print(f"\nAggregated BID data saved to {BID_OUTPUT}")
    print(f"Aggregated ASK data saved to {ASK_OUTPUT}")
    
    # Report any symbols that encountered errors
    if error_symbols:
        print(f"\nThe following symbols encountered errors and were skipped:")
        print(error_symbols)
        # Optionally, save the errored symbols to a CSV for reference
        pd.DataFrame(error_symbols, columns=["Symbol"]).to_csv("error_symbols_aggregation.csv", index=False)
    else:
        print("\nAll symbols were processed successfully without errors.")


try:
    aggregate_stock_data()
except Exception as e:
    print(f"Aggregation failed: {e}")

In [ ]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

# Define file paths
BID_FILE = "aggregated_bid.csv"
ASK_FILE = "aggregated_ask.csv"

# Load the aggregated CSVs into DataFrames
aggregated_bid_df = pd.read_csv(BID_FILE, parse_dates=['Date'])
aggregated_ask_df = pd.read_csv(ASK_FILE, parse_dates=['Date'])

# Set 'Date' as the index for easier handling
aggregated_bid_df.set_index('Date', inplace=True)
aggregated_ask_df.set_index('Date', inplace=True)

# Calculate the percentage of missing values for each stock in BID and ASK datasets
bid_missing_percentage = aggregated_bid_df.isna().mean() * 100  # Percentage per column
ask_missing_percentage = aggregated_ask_df.isna().mean() * 100  # Percentage per column

# Combine BID and ASK missing percentages into a single DataFrame for comparison
missing_data_summary = pd.DataFrame({
    'BID_Missing_Percentage': bid_missing_percentage,
    'ASK_Missing_Percentage': ask_missing_percentage
})

# Calculate the maximum missing percentage across BID and ASK for each stock
missing_data_summary['Max_Missing_Percentage'] = missing_data_summary[['BID_Missing_Percentage', 'ASK_Missing_Percentage']].max(axis=1)

# Display the summary
print("Missing Data Summary (First 5 Stocks):")
print(missing_data_summary.head())

# Visualize missing data in BID dataset
plt.figure(figsize=(12, 6))
sns.heatmap(aggregated_bid_df.isna(), cbar=False, cmap='viridis')
plt.title('Missing Data Heatmap - BID')
plt.show()

# Visualize missing data in ASK dataset
plt.figure(figsize=(12, 6))
sns.heatmap(aggregated_ask_df.isna(), cbar=False, cmap='viridis')
plt.title('Missing Data Heatmap - ASK')
plt.show()

# Define the missing data threshold (e.g., 5%)
MISSING_DATA_THRESHOLD = 15.0  # in percentage

# Identify stocks exceeding the threshold in either BID or ASK
stocks_to_remove = missing_data_summary[
    (missing_data_summary['BID_Missing_Percentage'] > MISSING_DATA_THRESHOLD) |
    (missing_data_summary['ASK_Missing_Percentage'] > MISSING_DATA_THRESHOLD)
].index.tolist()

print(f"\nNumber of stocks to remove: {len(stocks_to_remove)}")
print(f"Stocks to remove: {stocks_to_remove}")

# Remove identified stocks from both BID and ASK datasets
aggregated_bid_cleaned = aggregated_bid_df.drop(columns=stocks_to_remove)
aggregated_ask_cleaned = aggregated_ask_df.drop(columns=stocks_to_remove)

print(f"\nShape after removal - BID: {aggregated_bid_cleaned.shape}")
print(f"Shape after removal - ASK: {aggregated_ask_cleaned.shape}")

# Handle remaining missing data using forward fill
aggregated_bid_cleaned_ffill = aggregated_bid_cleaned.fillna(method='ffill')
aggregated_ask_cleaned_ffill = aggregated_ask_cleaned.fillna(method='ffill')

# Optionally, handle remaining NaNs with backward fill or interpolation
# aggregated_bid_cleaned_bfill = aggregated_bid_cleaned_ffill.fillna(method='bfill')
# aggregated_ask_cleaned_bfill = aggregated_ask_cleaned_ffill.fillna(method='bfill')
# aggregated_bid_cleaned_interp = aggregated_bid_cleaned_ffill.interpolate(method='linear')
# aggregated_ask_cleaned_interp = aggregated_ask_cleaned_ffill.interpolate(method='linear')

# Drop any remaining NaN values (optional)
aggregated_bid_final = aggregated_bid_cleaned_ffill.dropna()
aggregated_ask_final = aggregated_ask_cleaned_ffill.dropna()

print(f"\nFinal shape - BID: {aggregated_bid_final.shape}")
print(f"Final shape - ASK: {aggregated_ask_final.shape}")

# Define output file paths
CLEANED_BID_FILE = "aggregated_bid_cleaned.csv"
CLEANED_ASK_FILE = "aggregated_ask_cleaned.csv"

# Save the cleaned datasets
aggregated_bid_final.to_csv(CLEANED_BID_FILE)
aggregated_ask_final.to_csv(CLEANED_ASK_FILE)

print(f"\nCleaned BID data saved to {CLEANED_BID_FILE}")
print(f"Cleaned ASK data saved to {CLEANED_ASK_FILE}")


In [ ]:
# Define file paths
BID_CLEANED_FILE = "aggregated_bid_cleaned.csv"
ASK_CLEANED_FILE = "aggregated_ask_cleaned.csv"

# Load the cleaned CSVs into DataFrames
aggregated_bid_cleaned = pd.read_csv(BID_CLEANED_FILE, parse_dates=['Date'])
aggregated_ask_cleaned = pd.read_csv(ASK_CLEANED_FILE, parse_dates=['Date'])

# Visualize missing data in BID dataset
plt.figure(figsize=(12, 6))
sns.heatmap(aggregated_bid_cleaned.isna(), cbar=False, cmap='viridis')
plt.title('Missing Data Heatmap - BID Cleaned')
plt.show()

# Visualize missing data in ASK dataset
plt.figure(figsize=(12, 6))
sns.heatmap(aggregated_ask_cleaned.isna(), cbar=False, cmap='viridis')
plt.title('Missing Data Heatmap - ASK Cleaned')
plt.show()

In [ ]:
ld.close_session()